In [6]:
import requests
import json
import datetime
from pprint import pprint


In [7]:
!pip install pandas 


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import pandas as pd

In [9]:
import sys
sys.path.append("../")
from instaapi.local_config import * 

In [10]:
def basic_info():
    # 初期
    config = dict()
    #アクセストークン※facebook開発アプリから取得
    config["access_token"]         = ACCESS_TOKEN
    #アプリID※facebook開発アプリから取得
    config["app_id"]               = APP_ID
    #アプリシークレット※facebook開発アプリから取得
    config["app_secret"]           = APP_SECRET
    #インスタグラムビジネスアカウントID※facebook開発アプリから取得
    config['instagram_account_id'] = INSTAGRAM_ACCOUNT_ID
    #APIバージョン
    config["version"]              = 'v16.0'
    # 【修正不要】graphドメイン
    config["graph_domain"]         = 'https://graph.facebook.com/'
    # 【修正不要】エンドポイント
    config["endpoint_base"]        = config["graph_domain"]+config["version"] + '/'
    # 出力
    return config

In [11]:
# APIリクエスト用の関数
def InstaApiCall(url, params, request_type):
    
    # リクエスト
    if request_type == 'POST' :
        # POST
        req = requests.post(url,params)
    else :
        # GET
        req = requests.get(url,params)
    
    # レスポンス
    res = dict()
    res["url"] = url
    res["endpoint_params"]        = params
    res["endpoint_params_pretty"] = json.dumps(params, indent=4)
    res["json_data"]              = json.loads(req.content)
    res["json_data_pretty"]       = json.dumps(res["json_data"], indent=4)
    
    # 出力
    return res

In [12]:
def debugAT(params):
    # エンドポイントに送付するパラメータ
    Params = dict()
    Params["input_token"]  = params["access_token"]
    Params["access_token"] = params["access_token"]
    # エンドポイントURL
    url = params["graph_domain"] + "/debug_token"
    # 戻り値
    return InstaApiCall(url, Params, 'GET')

In [28]:
params

{'access_token': 'EAAKqUip6QXYBAJonvYsguNbvJDmZAAStZCUBkYnzAKPJb5Fqrp8ERfyyDQXjP4Sh1U3LhPj1oUAmTa7ZBAueXQ2feoLTSrJnRZAENKIHXQQogOMcgvO6moVHZBMHJqehMEJaxopgUxdbdTHDrHE1z3z28eKZC9hcn7dFYYfbjZALycbNI3pDXzP',
 'app_id': '750219830116726',
 'app_secret': 'd3f5378391eebcd12f9562fbd0cbb039',
 'instagram_account_id': '17841449282297673',
 'version': 'v16.0',
 'graph_domain': 'https://graph.facebook.com/',
 'endpoint_base': 'https://graph.facebook.com/v16.0/'}

In [31]:
response

{'url': 'https://graph.facebook.com/v16.0/17841449282297673/media',
 'endpoint_params': {'fields': 'id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,username',
  'access_token': 'EAAKqUip6QXYBAJonvYsguNbvJDmZAAStZCUBkYnzAKPJb5Fqrp8ERfyyDQXjP4Sh1U3LhPj1oUAmTa7ZBAueXQ2feoLTSrJnRZAENKIHXQQogOMcgvO6moVHZBMHJqehMEJaxopgUxdbdTHDrHE1z3z28eKZC9hcn7dFYYfbjZALycbNI3pDXzP'},
 'endpoint_params_pretty': '{\n    "fields": "id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,username",\n    "access_token": "EAAKqUip6QXYBAJonvYsguNbvJDmZAAStZCUBkYnzAKPJb5Fqrp8ERfyyDQXjP4Sh1U3LhPj1oUAmTa7ZBAueXQ2feoLTSrJnRZAENKIHXQQogOMcgvO6moVHZBMHJqehMEJaxopgUxdbdTHDrHE1z3z28eKZC9hcn7dFYYfbjZALycbNI3pDXzP"\n}',
 'json_data': {'error': {'message': 'Error validating access token: The session has been invalidated because the user changed their password or Facebook has changed the session for security reasons.',
   'type': 'OAuthException',
   'code': 190,
   'error_subcode': 460,
   'fbtra

In [13]:
# リクエスト
params   = basic_info()       # リクエストパラメータ
response = debugAT(params)    # レスポンス

# レスポンス
#pprint(response)

In [14]:
def GetPostId(params) :
    """
    ***********************************************************************************
    【APIエンドポイント】
    https://graph.facebook.com/{graph-api-version}/{ig-user-id}/insights?fields={fields}&access_token={access_token}
    ***********************************************************************************
    """
    # エンドポイントに渡すパラメータ
    Params = dict()
    Params['fields'] = 'id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,username' # インサイト指標                                       # 集計期間
    Params['access_token'] = params['access_token']                   # アクセストークン
    
    # エンドポイントURL
    url = params['endpoint_base'] + params['instagram_account_id'] + '/media' # endpoint url
    # 出力
    post_ids = []
    for i in InstaApiCall(url, Params, 'GET')['json_data']['data']:
      post_ids.append([i['id'],i['permalink']])
    return post_ids


In [30]:
GetPostId(params)

KeyError: 'data'

In [29]:
data = GetPostId(params)
print(data)

KeyError: 'data'

In [16]:
def PostInsights(params, period='day', media_id=111) :
    """
    ***********************************************************************************
    【APIエンドポイント】
    https://graph.facebook.com/{graph-api-version}/{ig-user-id}/insights?metric={metric}&period={period}
    ***********************************************************************************
    """
    # エンドポイントに渡すパラメータ
    Params = dict()
    Params['metric'] = 'carousel_album_engagement,carousel_album_impressions' # インサイト指標
    Params['period'] = period                                           # 集計期間
    Params['access_token'] = params['access_token']                   # アクセストークン
    
    # エンドポイントURL
    url = params['endpoint_base'] + str(media_id) + '/insights' # endpoint url
    # 出力
    return InstaApiCall(url, Params, 'GET')

In [17]:
# for d in data:
#   pprint(PostInsights(params, period='day',media_id=d[0])['json_data'])

In [18]:
def getUserMedia(params,pagingUrl='') :
    """ 
    ***************************************************************************************
    【APIのエンドポイント】
    https://graph.facebook.com/{graph-api-version}/{ig-user-id}/media?fields={fields}&access_token={access-token}
    ***************************************************************************************
    """
    # エンドポイントに渡すパラメータ
    Params = dict()
    Params['fields'] = 'id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,username' # フィールド
    Params['access_token'] = params['access_token']                                                 # アクセストークン
    
    if pagingUrl=='':
        # 先頭のページリンク取得
        url = params['endpoint_base'] + params['instagram_account_id'] + '/media'
    else :
        # 特定のページリンク取得
        url = pagingUrl
    
    # 出力
    return InstaApiCall(url, Params, 'GET')

In [19]:
response = getUserMedia(params)
response.keys()

dict_keys(['url', 'endpoint_params', 'endpoint_params_pretty', 'json_data', 'json_data_pretty'])

In [25]:
response

{'url': 'https://graph.facebook.com/v16.0/17841449282297673/media',
 'endpoint_params': {'fields': 'id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,username',
  'access_token': 'EAAKqUip6QXYBAJonvYsguNbvJDmZAAStZCUBkYnzAKPJb5Fqrp8ERfyyDQXjP4Sh1U3LhPj1oUAmTa7ZBAueXQ2feoLTSrJnRZAENKIHXQQogOMcgvO6moVHZBMHJqehMEJaxopgUxdbdTHDrHE1z3z28eKZC9hcn7dFYYfbjZALycbNI3pDXzP'},
 'endpoint_params_pretty': '{\n    "fields": "id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,username",\n    "access_token": "EAAKqUip6QXYBAJonvYsguNbvJDmZAAStZCUBkYnzAKPJb5Fqrp8ERfyyDQXjP4Sh1U3LhPj1oUAmTa7ZBAueXQ2feoLTSrJnRZAENKIHXQQogOMcgvO6moVHZBMHJqehMEJaxopgUxdbdTHDrHE1z3z28eKZC9hcn7dFYYfbjZALycbNI3pDXzP"\n}',
 'json_data': {'error': {'message': 'Error validating access token: The session has been invalidated because the user changed their password or Facebook has changed the session for security reasons.',
   'type': 'OAuthException',
   'code': 190,
   'error_subcode': 460,
   'fbtra

In [26]:
for i, post in enumerate(response['json_data']['data']) :
    print(post)

KeyError: 'data'

In [22]:

print ("\n----------"+str(response['json_data']['data'][0]["username"])+"の投稿内容 ----------\n")
for i, post in enumerate(response['json_data']['data']) :
    print ("\n----------投稿内容("+str(i+1)+")----------\n")
    print ("投稿日: " + post['timestamp'])
    print ("投稿メディアID: "+post['id'])
    print ("メディア種別: " + post['media_type'])
    print ("投稿リンク: " + post['permalink'])
    print ("\n投稿文: "  + post['caption'])
    

KeyError: 'data'

In [23]:
# インスタグラム運用アカウントのインサイトを確認
def UserInsights(params, period='day') :
    """
    ***********************************************************************************
    【APIエンドポイント】
    https://graph.facebook.com/{graph-api-version}/{ig-user-id}/insights?metric={metric}&period={period}
    ***********************************************************************************
    """
    # エンドポイントに渡すパラメータ
    Params = dict()
    Params['metric'] = 'follower_count,impressions,profile_views,reach, get_directions_clicks, text_message_clicks, website_clicks, email_contacts, phone_call_clicks' # インサイト指標
    Params['period'] = period                                           # 集計期間
    Params['access_token'] = params['access_token']                     # アクセストークン
    
    # エンドポイントURL
    url = params['endpoint_base'] + params['instagram_account_id'] + '/insights' # endpoint url
    
    # 出力
    return InstaApiCall(url, Params, 'GET')

In [24]:
# メディア期間
period = 'day'

# API呼出
params   = basic_info()
response = UserInsights(params, period)["json_data"]["data"]

# 結果出力 
for insight in response:
    print (insight['title'] + "（" + insight['description'] + ")" + " (" + insight['period'] + ")") 
    for value in insight['values'] : # loop over each value
        print ("\t" + value['end_time'] + ": " + str( value['value'] )) # print out counts for the date

KeyError: 'data'

In [18]:
import requests
#import pandas as pd
#pd.set_option('display.max_rows', None)

# アクセス情報
business_account_id = params['instagram_account_id']
token = params['access_token']
username = 'kgt_clothing'
fields = 'name,username,biography,follows_count,followers_count,media_count'
media_fields = 'timestamp,like_count,comments_count,caption'


# ユーザー情報を取得する
def user_info(business_account_id,token,username,fields):
    request_url = "https://graph.facebook.com/v11.0/{business_account_id}?fields=business_discovery.username({username}){{{fields}}}&access_token={token}".format(business_account_id=business_account_id,username=username,fields=fields,token=token)
#     print(request_url)
    response = requests.get(request_url)
    return response.json()['business_discovery']

#print(user_info(business_account_id,token,username,fields))

# メディア情報を取得する
def user_media_info(business_account_id,token,username,media_fields):
    all_response = []

    request_url = "https://graph.facebook.com/v11.0/{business_account_id}?fields=business_discovery.username({username}){{media{{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token)
#     print(request_url)
    response = requests.get(request_url)
    result = response.json()['business_discovery']
    
    all_response.append(result['media']['data'])
    
    # 過去分がある場合は過去分全て取得する(1度に取得できる件数は25件)
    if 'after' in result['media']['paging']['cursors'].keys():
        next_token = result['media']['paging']['cursors']['after']
        while next_token is not None:
            request_url = "https://graph.facebook.com/v11.0/{business_account_id}?fields=business_discovery.username({username}){{media.after({next_token}){{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token,next_token=next_token)
#             print(request_url)
            response = requests.get(request_url)
            result = response.json()['business_discovery']
            all_response.append(result['media']['data'])
            if 'after' in result['media']['paging']['cursors'].keys():
                next_token = result['media']['paging']['cursors']['after']
            else:
                next_token = None
    
    return all_response

result = user_media_info(business_account_id,token,username,media_fields)

In [18]:
result

[[{'timestamp': '2023-04-25T09:30:00+0000',
   'like_count': 33,
   'comments_count': 1,
   'caption': '---\n------\n◽︎Jacket : #unitedarrows \n◽︎Shirts : #kamakurashirts \n◽︎Tie : #beamsf \n------\n\n#mensfashion #mensstyle #menswear #mensfashionpost #styling #ootd #ootdmens\n#mensclothing #menslook #fashionstyle #ファッションスナップ #ファッションポスト #メンズスナップ #ビジネスコーデ #オフィスコーデ \n#スーツスタイル #メンズスーツ\n#ドレスコーデ #メンズドレス\n#スーツコーデ #ドレスファッション',
   'id': '18358608613016133'},
  {'timestamp': '2023-04-24T09:30:00+0000',
   'like_count': 61,
   'comments_count': 0,
   'caption': '---\n------\n◽︎Suits : #unitedarrows #greenlabelrelaxing \n◽︎Shirts : #kamakurashirts \n◽︎Tie : #gierremilano \n◽︎Watch : #rolex #rolexexplorer1\n------\n\n#mensfashion #mensstyle #menswear #mensfashionpost #styling #ootd #ootdmens\n#mensclothing #menslook #fashionstyle #ファッションスナップ #ファッションポスト #メンズスナップ #ビジネスコーデ #オフィスコーデ \n#スーツスタイル #メンズスーツ\n#ドレスコーデ #メンズドレス\n#スーツコーデ #ドレスファッション',
   'id': '18083207689341792'},
  {'timestamp': '2023-04-23T09: